In [1]:
import numpy as np
import open3d as o3d
import os
import zipfile
from sklearn.neighbors import NearestNeighbors
import cv2 as cv
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
import json

In [2]:
# Уравнение наклонной прямой
def lin_equ(l1, l2):
    """Line encoded as l=(x,y)."""
    m = (l2[1] - l1[1]) / (l2[0] - l1[0])
    c = (l2[1] - (m * l2[0]))
    return m, c

In [22]:
#path="point_cloud_gt/clouds_tof/cloud_0_0047.pcd" # один
#path="point_cloud_gt/clouds_tof/cloud_0_0150.pcd" # трое
#path="point_cloud_gt/clouds_tof/cloud_0_0138.pcd" # толпа
folder = "point_cloud_gt/"
path=folder + "cloud_0_0068.pcd" # толпа

In [32]:
path="cloud_0_1620667033984658.pcd"
print(path)

cloud_0_1620667033984658.pcd


In [10]:
def write_json(boxes, classes, path):    
    json_dict = dict()
    json_dict["figures"] = list()
    
    for i, bb in enumerate(boxes):
        position = bb.center
        dimensions = bb.extent
        r = Rotation.from_matrix(np.hstack((bb.R[:,1:], bb.R[:,:1])))
        angle = r.as_euler('xyz').round(5)
        cls = classes[i]

        json_dict["figures"].append(dict())

        json_dict["figures"][-1]["geometryType"] = "cuboid_3d"

        json_dict["figures"][-1]["objectKey"] = cls

        json_dict["figures"][-1]["geometry"] = dict()
        json_dict["figures"][-1]["geometry"]["position"] = {"x": position[0],
                                                            "y": position[1],
                                                            "z": position[2]}

        json_dict["figures"][-1]["geometry"]["dimensions"] = {"x": dimensions[1],
                                                              "y": dimensions[2],
                                                              "z": dimensions[0]}

        json_dict["figures"][-1]["geometry"]["rotation"] = {"x": angle[0],
                                                            "y": angle[1],
                                                            "z": angle[2]}
        json_dict["figures"][-1]["door"] = "unknown"
    
    with open(path + '.json', 'w') as f:
        json.dump(json_dict, f, indent=4)

In [34]:
pcd = o3d.io.read_point_cloud(path)
pcd_colors = np.asarray(pcd.colors)
pcd_points = np.asarray(pcd.points)
# print(pcd_points.shape)
mesh_box1 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                height=.005,
                                                depth=4).translate((-1, -0.22, 2))

R = mesh_box1.get_rotation_matrix_from_xyz((0.0, 0.0,0.01))
mesh_box1.rotate(R, center=(-1, -0.22, 3))

mesh_box2 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                height=.005,
                                                depth=1.5).translate((-1, -0.36, 4.5))

R = mesh_box2.get_rotation_matrix_from_xyz((-0.07, 0, 0))
mesh_box2.rotate(R, center=(-1, -0.36, 3)).translate((0, -0.08, 0))


mesh_box3 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                height=.005,
                                                depth=2.5).translate((-1, -0.36, 1.5))

R = mesh_box3.get_rotation_matrix_from_xyz((-0.15, 0, 0))
mesh_box3.rotate(R, center=(-1, -0.36, 3)).translate((0, -0.125, 0.51))


mesh_box1.compute_vertex_normals()
mesh_box1.paint_uniform_color([0.1, 0.3, 0.1])

mesh_box2.compute_vertex_normals()
mesh_box2.paint_uniform_color([0.2, 0.5, 0.3])

mesh_box3.compute_vertex_normals()
mesh_box3.paint_uniform_color([0.4, 0.1, 0.1])

# Example Usage:
y1=np.asarray(mesh_box3.vertices)[0,1]
z1=np.asarray(mesh_box3.vertices)[0,2]
y2=np.asarray(mesh_box3.vertices)[-1,1]
z2=np.asarray(mesh_box3.vertices)[-1,2]

factors=lin_equ((y1, z1,), (y2, z2))
factors

pcd = o3d.io.read_point_cloud(path)
np.asarray(pcd.colors)
pcd_colors = np.asarray(pcd.colors)
pcd_points = np.asarray(pcd.points)

# print(pcd_points.shape)
neigh = NearestNeighbors(n_neighbors=3, radius=0.4)
neigh.fit(pcd_points)
distances, indices = neigh.kneighbors(pcd_points)
distances = distances.max(axis=1).reshape(distances.shape[0])
pcd_points = pcd_points[distances < 0.004]
pcd_colors = pcd_colors[distances < 0.004]


pcd.points = o3d.utility.Vector3dVector(pcd_points)
pcd_colors = np.asarray(pcd.colors)
pcd_points = np.asarray(pcd.points)

condition_1 = pcd_points[:,1] < np.asarray(mesh_box1.vertices)[:,1].min() # Условие, что точки слева от горизонтальной плоскости
condition_3 = pcd_points[:,2] < (pcd_points[:,1]*factors[0] + factors[1] - 0.2 ) # Условие, что точки справа от наклонной плоскости
condition_4 = pcd_points[:,2] > (pcd_points[:,1]*factors[0] + factors[1] - 1.6 )
pcd_points=pcd_points[condition_1 & condition_3 & condition_4]
pcd_new = o3d.geometry.PointCloud()
pcd_new.points = o3d.utility.Vector3dVector(pcd_points)

a = np.asarray(pcd.points)
#Только перон
peron_points = a[(a[:,0]<0.95) & (a[:,0]>-0.95) & (a[:,1]<0.75) & (a[:,1]>-0.2) & (a[:,2]<7) & (a[:,2]>5.3)]
hole = peron_points[:, :2]

plt.figure(figsize = (14,10))
plt.scatter(peron_points[:, 0], peron_points[:, 1])
plt.axis('off')
plt.savefig('temp.jpg', bbox_inches='tight')
plt.close()

cut = ((40,510),(40,750))
im = cv.imread('temp.jpg')[cut[0][0]:cut[0][1], cut[1][0]:cut[1][1]]
imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

peron_cubes = []
for cont in contours:
    try:
        min_value = cont.squeeze(1).min(axis=0)
        max_value = cont.squeeze(1).max(axis=0)
        left = min_value[0]/(cut[1][1]-cut[1][0])*1.9 - 0.95
        bottom = .76 - min_value[1]/(cut[0][1]-cut[0][0])*0.95
        right = max_value[0]/(cut[1][1]-cut[1][0])*1.9 - .95
        top = .76 - max_value[1]/(cut[0][1]-cut[0][0])*0.95       
        area = (right-left) * (bottom-top) 
        if area<0.06:
            continue
        z = a[(a[:,0]<right) & (a[:,0]>left) & (a[:,1]<bottom) & (a[:,1]>top)]
        z = min(z[:,2])
        mesh_box4 = o3d.geometry.TriangleMesh.create_box(width=right-left,
                                                        height=bottom-top,
                                                        depth=5.4-z).translate((left, top, z))
        aabb2 = mesh_box4.get_oriented_bounding_box()
        peron_cubes.append(aabb2)
    except:
        continue

if np.asarray(pcd_new.points).shape[0] > 20:
    aabb = pcd_new.get_oriented_bounding_box()           
    o3d.visualization.draw_geometries([aabb,pcd]+ peron_cubes)
    if np.asarray(pcd_new.points).shape[0] <= 50:
        obj = 'empty'
    elif (np.asarray(pcd_new.points).shape[0] > 50) and (np.asarray(pcd_new.points).shape[0] < 1000):
        obj = 'wear'
    else: obj = 'human'
    write_json([aabb] + peron_cubes, [obj] + ['human'] * len(peron_cubes), path)
else:
    o3d.visualization.draw_geometries([pcd] + peron_cubes)
    write_json(peron_cubes, ['human'] * len(peron_cubes), path)  

(239878, 3)
(239878, 3)


In [ ]:
lstfile = os.listdir("clouds_tof")
common_path="clouds_tof"
for i,f in enumerate(lstfile):
#     print(f)
        path="clouds_tof/"+f+"/"+f
        pcd = o3d.io.read_point_cloud(path)
        pcd_colors = np.asarray(pcd.colors)
        pcd_points = np.asarray(pcd.points)
        # print(pcd_points.shape)
        mesh_box1 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                        height=.005,
                                                        depth=4).translate((-1, -0.22, 2))

        R = mesh_box1.get_rotation_matrix_from_xyz((0.0, 0.0,0.01))
        mesh_box1.rotate(R, center=(-1, -0.22, 3))

        mesh_box2 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                        height=.005,
                                                        depth=1.5).translate((-1, -0.36, 4.5))

        R = mesh_box2.get_rotation_matrix_from_xyz((-0.07, 0, 0))
        mesh_box2.rotate(R, center=(-1, -0.36, 3)).translate((0, -0.08, 0))


        mesh_box3 = o3d.geometry.TriangleMesh.create_box(width=2,
                                                        height=.005,
                                                        depth=2.5).translate((-1, -0.36, 1.5))

        R = mesh_box3.get_rotation_matrix_from_xyz((-0.15, 0, 0))
        mesh_box3.rotate(R, center=(-1, -0.36, 3)).translate((0, -0.125, 0.51))


        mesh_box1.compute_vertex_normals()
        mesh_box1.paint_uniform_color([0.1, 0.3, 0.1])

        mesh_box2.compute_vertex_normals()
        mesh_box2.paint_uniform_color([0.2, 0.5, 0.3])

        mesh_box3.compute_vertex_normals()
        mesh_box3.paint_uniform_color([0.4, 0.1, 0.1])

        # Example Usage:
        y1=np.asarray(mesh_box3.vertices)[0,1]
        z1=np.asarray(mesh_box3.vertices)[0,2]
        y2=np.asarray(mesh_box3.vertices)[-1,1]
        z2=np.asarray(mesh_box3.vertices)[-1,2]

        factors=lin_equ((y1, z1,), (y2, z2))
        factors

        pcd = o3d.io.read_point_cloud(path)
        np.asarray(pcd.colors)
        pcd_colors = np.asarray(pcd.colors)
        pcd_points = np.asarray(pcd.points)

        # print(pcd_points.shape)
        neigh = NearestNeighbors(n_neighbors=3, radius=0.4)
        neigh.fit(pcd_points)
        distances, indices = neigh.kneighbors(pcd_points)
        distances = distances.max(axis=1).reshape(distances.shape[0])
        pcd_points = pcd_points[distances < 0.004]
        pcd_colors = pcd_colors[distances < 0.004]


        pcd.points = o3d.utility.Vector3dVector(pcd_points)
        pcd_colors = np.asarray(pcd.colors)
        pcd_points = np.asarray(pcd.points)

        condition_1 = pcd_points[:,1] < np.asarray(mesh_box1.vertices)[:,1].min() # Условие, что точки слева от горизонтальной плоскости
        condition_3 = pcd_points[:,2] < (pcd_points[:,1]*factors[0] + factors[1] - 0.2 ) # Условие, что точки справа от наклонной плоскости
        condition_4 = pcd_points[:,2] > (pcd_points[:,1]*factors[0] + factors[1] - 1.6 )
        pcd_points=pcd_points[condition_1 & condition_3 & condition_4]
        pcd_new = o3d.geometry.PointCloud()
        pcd_new.points = o3d.utility.Vector3dVector(pcd_points)

        a = np.asarray(pcd.points)
        #Только перон
        peron_points = a[(a[:,0]<0.95) & (a[:,0]>-0.95) & (a[:,1]<0.75) & (a[:,1]>-0.2) & (a[:,2]<7) & (a[:,2]>5.3)]
        hole = peron_points[:, :2]

        plt.figure(figsize = (14,10))
        plt.scatter(peron_points[:, 0], peron_points[:, 1])
        plt.axis('off')
        plt.savefig('temp.jpg', bbox_inches='tight')
        plt.close()

        cut = ((40,510),(40,750))
        im = cv.imread('temp.jpg')[cut[0][0]:cut[0][1], cut[1][0]:cut[1][1]]
        imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(imgray, 127, 255, 0)
        contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        peron_cubes = []
        for cont in contours:
            try:
                min_value = cont.squeeze(1).min(axis=0)
                max_value = cont.squeeze(1).max(axis=0)
                left = min_value[0]/(cut[1][1]-cut[1][0])*1.9 - 0.95
                bottom = .76 - min_value[1]/(cut[0][1]-cut[0][0])*0.95
                right = max_value[0]/(cut[1][1]-cut[1][0])*1.9 - .95
                top = .76 - max_value[1]/(cut[0][1]-cut[0][0])*0.95       
                area = (right-left) * (bottom-top) 
                if area<0.06:
                    continue
                z = a[(a[:,0]<right) & (a[:,0]>left) & (a[:,1]<bottom) & (a[:,1]>top)]
                z = min(z[:,2])
                mesh_box4 = o3d.geometry.TriangleMesh.create_box(width=right-left,
                                                                height=bottom-top,
                                                                depth=5.4-z).translate((left, top, z))
                aabb2 = mesh_box4.get_oriented_bounding_box()
                peron_cubes.append(aabb2)
            except:
                continue

        if np.asarray(pcd_new.points).shape[0] > 20:
            aabb = pcd_new.get_oriented_bounding_box()           
#             o3d.visualization.draw_geometries([aabb,pcd]+ peron_cubes)
            if np.asarray(pcd_new.points).shape[0] <= 50:
                obj = 'empty'
            elif (np.asarray(pcd_new.points).shape[0] > 50) and (np.asarray(pcd_new.points).shape[0] < 1000):
                obj = 'wear'
            else: obj = 'human'
            write_json([aabb] + peron_cubes, [obj] + ['human'] * len(peron_cubes), path)
        else:
#             o3d.visualization.draw_geometries([pcd] + peron_cubes)
            write_json(peron_cubes, ['human'] * len(peron_cubes), path)  
